In [ ]:
!pip install mysql-connector-python influxdb

In [1]:
!docker-compose up -d

time="2025-04-05T18:24:31+02:00" level=warning msg="C:\\Users\\dalli\\Big-Data-Gruppe-5\\docker-compose.yml: the attribute `version` is obsolete, it will be ignored, please remove it to avoid potential confusion"
 mysql Pulling 
 influxdb Pulling 
 influxdb Pulled 
 c11056354384 Pulling fs layer 
 cb8acbf2440c Pulling fs layer 
 cea172a6e83b Pulling fs layer 
 49978e7ccddf Pulling fs layer 
 79f239a40e62 Pulling fs layer 
 548990e33276 Pulling fs layer 
 b2ead3e96e6b Pulling fs layer 
 daac2c594bdd Pulling fs layer 
 fae51f7de1fb Pulling fs layer 
 769c3ac51f88 Pulling fs layer 
 cea172a6e83b Downloading [=>                                                 ]  1.049MB/49.09MB
 c11056354384 Download complete 
 cea172a6e83b Downloading [=>                                                 ]  1.049MB/49.09MB
 cea172a6e83b Downloading [=>                                                 ]  1.049MB/49.09MB
 548990e33276 Download complete 
 daac2c594bdd Download complete 
 cea172a6e83b Downloadin

In [2]:
import pandas as pd
import mysql.connector
from influxdb import InfluxDBClient
from influxdb_client import InfluxDBClient, Point, WriteOptions
import yfinance as yf

In [3]:
# Daten mit der Yahoo Finance API abrufen und in csv files speichern

def get_data(ticker, start_date, end_date, interval):

    data = yf.download(ticker, start=start_date, end=end_date, interval=interval)
    data = data[['Close', 'Volume']].reset_index()

    # Spalten umbenennen
    data.columns = ['Date', 'Closing Price', 'Volume']
    return data

def save_to_csv(data, filename):
    data.to_csv(filename, index=False)



tickers = ['BTC-USD', 'ETH-USD', 'SOL-USD', 'BNB-USD', 'DOGE-USD', 'ADA-USD', 'TRX-USD', 'XRP-USD', 'LINK-USD']
start_date = '2016-01-01'
end_date = '2025-04-01'
interval = '1d'

for ticker in tickers:
    data = get_data(ticker, start_date, end_date, interval)
    coin_name = ticker.split('-')[0]
    filename = f"yahoo/{coin_name}_2022-2024.csv"
    save_to_csv(data, filename)
    print(f"Data for {ticker} saved to {filename}")


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


Data for BTC-USD saved to yahoo/BTC_2022-2024.csv


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Data for ETH-USD saved to yahoo/ETH_2022-2024.csv
Data for SOL-USD saved to yahoo/SOL_2022-2024.csv


...

Data for DOGE-USD saved to yahoo/DOGE_2022-2024.csv
Data for ADA-USD saved to yahoo/ADA_2022-2024.csv



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Data for TRX-USD saved to yahoo/TRX_2022-2024.csv
Data for XRP-USD saved to yahoo/XRP_2022-2024.csv


[*********************100%***********************]  1 of 1 completed

Data for LINK-USD saved to yahoo/LINK_2022-2024.csv


In [6]:
bucket = "coins"
org = "my-org"
token = "my-secret-token"

# Erstelle den InfluxDB-Client
client = InfluxDBClient(url="http://localhost:8086", token=token, org=org)
write_api = client.write_api(write_options=WriteOptions(batch_size=1000))


buckets_api = client.buckets_api()
bucket_obj = buckets_api.find_bucket_by_name(bucket)
if not bucket_obj:
    buckets_api.create_bucket(bucket_name=bucket, org=org)
    print(f"Bucket '{bucket}' created.")
else:
    print(f"Bucket '{bucket}' already exists.")

Bucket 'coins' created.


In [7]:
# safe csv files in Influx


coins = ['BTC', 'ETH', 'SOL', 'BNB', 'DOGE', 'ADA', 'TRX', 'XRP', 'LINK']


for coin in coins:
    csv_file = f"yahoo/{coin}_2022-2024.csv"
    df = pd.read_csv(csv_file)
    df['Date'] = pd.to_datetime(df['Date'])

    json_body = []
    for _, row in df.iterrows():
        json_body.append({
            "measurement": coin,
            "time": row["Date"].isoformat(),
            "fields": {
                "closing_price": float(row["Closing Price"]),
                "volume": int(row["Volume"])
            }
        })


    write_api.write(bucket=bucket, org=org, record=json_body)
    print(f"InfluxDB import for {coin} done.")


InfluxDB import for BTC done.
InfluxDB import for ETH done.
InfluxDB import for SOL done.
InfluxDB import for BNB done.
InfluxDB import for DOGE done.
InfluxDB import for ADA done.
InfluxDB import for TRX done.
InfluxDB import for XRP done.
InfluxDB import for LINK done.


In [5]:
# save csv files in MySQL

mysql_config = {
    "host": "localhost",
    "user": "root",
    "password": "yourpassword",
    "database": "kryptodb"
}

coins = ['BTC', 'ETH', 'SOL', 'BNB', 'DOGE', 'ADA', 'TRX', 'XRP', 'LINK']

for coin in coins:
    mysql_conn = mysql.connector.connect(**mysql_config)
    cursor = mysql_conn.cursor()
    csv_file = f"yahoo/{coin}_2022-2024.csv"
    df = pd.read_csv(csv_file)

    df['Date'] = pd.to_datetime(df['Date'])

    create_table_sql = f"""
    CREATE TABLE IF NOT EXISTS {coin.lower()} (
        date DATE PRIMARY KEY,
        closing_price FLOAT,
        volume BIGINT
    );
    """
    cursor.execute(create_table_sql)

    for _, row in df.iterrows():
        insert_sql = f"""
        REPLACE INTO {coin.lower()} (date, closing_price, volume)
        VALUES (%s, %s, %s);
        """
        cursor.execute(insert_sql, (row['Date'].date(), row['Closing Price'], row['Volume']))

    mysql_conn.commit()
    cursor.close()
    mysql_conn.close()
    print(f"MySQL import for {coin} done.")


MySQL import for BTC done.
MySQL import for ETH done.
MySQL import for SOL done.
MySQL import for BNB done.
MySQL import for DOGE done.
MySQL import for ADA done.
MySQL import for TRX done.
MySQL import for XRP done.
MySQL import for LINK done.
